In [68]:
from nltk.stem import WordNetLemmatizer
#from functools32 import lru_cache
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk

def tokens(sentence):
  return word_tokenize(sentence)

def lemmatize(tokens):
  wnl = WordNetLemmatizer()
  return [wnl.lemmatize(token) for token in tokens]

def postags(tokens):
  return nltk.pos_tag(tokens)

In [69]:
sentence = "My name is Daniel"
word_tokens = tokens(sentence)
print lemmatize(word_tokens)
print postags(word_tokens)

import glob
import re

fname = "../STS-data/STS2013-gold/STS.input.headlines.txt"
with open(fname) as f:
    lines = f.readlines()

Data = []
    
for fname in glob.glob("../STS-data/*201[1-5]*/*input*.txt"):
    print fname
    with open(fname) as f:
        lines = f.readlines()
        for line in lines:
            if len(line.split('\t')) !=2 :
                print fname
                break;
            else:
                Data.append(line.split('\t'))
for fname in glob.glob("../STS-data/*2016*/*input*.txt"):
    print fname
    print re.sub("input","gs",fname)
    with open(fname) as f:
        lines = f.readlines()
        with open(re.sub("input","gs",fname)) as nfile:
            nlines = nfile.readlines()
        #nlines[index]
        for index,line in enumerate(lines):
            if len(line.split('\t')) !=4 :
                #print fname
                break;
            else:
                x = line.split('\t')
                Data.append([x[0],x[1],nlines[index]])
Data[-1]

['My', 'name', 'is', 'Daniel']
[('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Daniel', 'NNP')]
../STS-data/STS2012-gold/STS.input.MSRpar.txt
../STS-data/STS2012-gold/STS.input.MSRvid.txt
../STS-data/STS2012-gold/STS.input.SMTeuroparl.txt
../STS-data/STS2012-gold/STS.input.surprise.OnWN.txt
../STS-data/STS2012-gold/STS.input.surprise.SMTnews.txt
../STS-data/STS2012-train/STS.input.MSRpar.txt
../STS-data/STS2012-train/STS.input.MSRvid.txt
../STS-data/STS2012-train/STS.input.SMTeuroparl.txt
../STS-data/STS2013-gold/STS.input.FNWN.txt
../STS-data/STS2013-gold/STS.input.headlines.txt
../STS-data/STS2013-gold/STS.input.OnWN.txt
../STS-data/STS2014-gold/STS.input.deft-forum.txt
../STS-data/STS2014-gold/STS.input.deft-news.txt
../STS-data/STS2014-gold/STS.input.headlines.txt
../STS-data/STS2014-gold/STS.input.images.txt
../STS-data/STS2014-gold/STS.input.OnWN.txt
../STS-data/STS2014-gold/STS.input.tweet-news.txt
../STS-data/STS2015-gold/STS.input.answers-forums.txt
../STS-data/STS2015-gold/

["Is it inappropriate to spend personal resources for the company's benefit?",
 "Is it ethical/legal to use the University's resources for a personal project?",
 '\n']

In [70]:
Data = []
fname = "../STS-data/STS2013-gold/STS.input.headlines.txt"
with open(fname) as f:
    lines = f.readlines()

    with open(re.sub("input","gs",fname)) as nfile:
        nlines = nfile.readlines()
        for index,line in enumerate(lines):
            if len(line.split('\t')) !=2 :
                #print fname
                break;
            else:
                if(re.search("(\d)((\.)(\d))?", nlines[index])):
                    ;#print nlines[index]
                else:
                    print lines[index]
                    break
                x = line.split('\t')
                Data.append([x[0],x[1], float(re.sub("\n","",nlines[index]))])
                
#print Data

In [71]:
Data = []
fname = "../STS-data/sts2016-english-with-gs-v1.0/STS2016.input.question-question.txt"
with open(fname) as f:
    lines = f.readlines()

    with open(re.sub("input","gs",fname)) as nfile:
        nlines = nfile.readlines()
        #print nlines
        
        for index,line in enumerate(lines):
            if len(line.split('\t')) !=4 :
                #print fname
                break;
            else:
                if(not re.search("(\d)((\.)(\d))?", nlines[index])):
                    continue;#print nlines[index]
                #else:
                    #print lines[index]
                    #continue
                x = line.split('\t')
                Data.append([x[0],x[1], float(re.sub("\n","",nlines[index]))])
                
#print Data

In [72]:
from collections import defaultdict
from collections import Counter
import preprocess as process
import utilities as utility
import math

def ngram_vector_keys(tokens, ngram_size=1):
  vector_keys = []
  for i in range(len(tokens)-(ngram_size-1)):
    vector_keys.append(tuple(tokens[i:i+(ngram_size)]))
    # vector_keys.append(" ".join(tokens[i:i+(ngram_size)]))
  return vector_keys

def containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1):
  set1 = set(ngram_vector_keys(sent_1_tokens, ngram_size))
  set2 = set(ngram_vector_keys(sent_2_tokens, ngram_size))
  containment_of_sentence_1_in_2 = float(len(set1.intersection(set2)))/len(set1)
  containment_of_sentence_2_in_1 = float(len(set1.intersection(set2)))/len(set2)
  return containment_of_sentence_1_in_2, containment_of_sentence_2_in_1

def JaccardCoefficient(sent_1_tokens, sent_2_tokens, ngram_size=1):
  set1 = set(ngram_vector_keys(sent_1_tokens, ngram_size))
  set2 = set(ngram_vector_keys(sent_2_tokens, ngram_size))
  #print set1,set2
  return float(len(set1.intersection(set2)))/len(set1.union(set2))

def TFIDF(documents):
  Vocabulary = Counter()
  DocVectors = []
  IDFVector = Counter()
  No_of_Documents = float(len(documents))
  for document in documents:
    tf_single_doc_count = Counter(process.tokens(document))
    Vocabulary+= tf_single_doc_count
    DocVectors.append(tf_single_doc_count)
    IDFVector += Counter(tf_single_doc_count.keys())
  # print IDFVector
  for key in IDFVector.keys():
    IDFVector[key] = math.log(No_of_Documents/(1+IDFVector[key]))
  # print IDFVector
  # IDFVector = defaultdict(lambda:0.0, dict((key,Vocabulary[key]*) for key in c.keys()))
  TFIDFScores = defaultdict(lambda:0.0, dict((key,Vocabulary[key]*IDFVector[key]) for key in Vocabulary.keys()))
  # print TFIDFScores, IDFVector, Vocabulary
  return TFIDFScores, Vocabulary, DocVectors, IDFVector

def DocvectorTFIDF(TFIDFScores, tokens):
  return defaultdict(lambda:0.0, dict((key,TFIDFScores[key] if key in TFIDFScores.keys() else 0.0) for key in tokens))


def cosinesimilarity(document1, document2, TFIDFScores):
  tokens1 = set(process.tokens(document1))
  tokens2 = set(process.tokens(document2))
  vector1 = DocvectorTFIDF(TFIDFScores, tokens1)
  vector2 = DocvectorTFIDF(TFIDFScores, tokens2)
  len_vector_1 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_1
  len_vector_2 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_2
  print utility.dict_dotprod(vector1,vector2)
  cosine_similarity_score = (utility.dict_dotprod(vector1,vector2))/float((len_vector_2*len_vector_1))
  return cosine_similarity_score


def cosinesimilarity_without_TFIDF(document1, document2):
  vector1 = Counter(process.tokens(document1))
  vector2 = Counter(process.tokens(document2))
  print vector1
  print vector2
  len_vector_1 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_1
  len_vector_2 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_2
  cosine_similarity_score = (utility.dict_dotprod(vector1,vector2))/float((len_vector_2*len_vector_1))
  return cosine_similarity_score


In [73]:
TFIDFScores, Vocabulary, DocVectors, IDFVector = TFIDF(["i ike", "i ike"])
TFIDFScores = {"i":1,"ike":1}
v = DocvectorTFIDF(TFIDFScores, process.tokens("i ike"))
#print v
#print utility.dict_dotprod(v,DocvectorTFIDF(TFIDFScores, process.tokens("i tes")))
print cosinesimilarity("i ike", "i tes", {"i":1,"ike":1})
print cosinesimilarity_without_TFIDF("i ike", "i ike")

JaccardCoefficient(["sent_1_tokens","test"], ["sent_2_tokens","tes","Tesd","Tesd","Tesd"], ngram_size=2)
print containment_coefficienct(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=2)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=3)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=4)

1.41421356237
1.41421356237
1.0
0.5
Counter({'i': 1, 'ike': 1})
Counter({'i': 1, 'ike': 1})
1.41421356237
1.41421356237
1.0
(1.0, 0.6)
0.6
0.5
0.428571428571
0.125


In [74]:
import numpy
def ngram_vector_keys(tokens, ngram_size=1):
  vector_keys = []
  for i in range(len(tokens)-(ngram_size-1)):
    vector_keys.append(tuple(tokens[i:i+(ngram_size)]))
    # vector_keys.append(" ".join(tokens[i:i+(ngram_size)]))
  return vector_keys

def character_ngram_vector_keys(tokens, ngram_size=2):
  vector_keys = []
  for token in tokens:
    for i in range(len(token)-(ngram_size-1)):
      vector_keys.append(tuple(token[i:i+(ngram_size)]))
  return vector_keys

def ngram_keys(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram):
  if character_ngram:
    set1 = set(character_ngram_vector_keys(sent_1_tokens, ngram_size))
    set2 = set(character_ngram_vector_keys(sent_2_tokens, ngram_size))
  else:
    set1 = set(ngram_vector_keys(sent_1_tokens, ngram_size))
    set2 = set(ngram_vector_keys(sent_2_tokens, ngram_size))
  print set1,set2
  return set1, set2

def containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1, character_ngram=False):
  set1, set2 = ngram_keys(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram)
  intersecting_keys_len = len(set1.intersection(set2))
  containment_of_sentence_1_in_2 = float(intersecting_keys_len)/len(set1)
  containment_of_sentence_2_in_1 = float(intersecting_keys_len)/len(set2)
  # return containment_of_sentence_1_in_2, containment_of_sentence_2_in_1
  return numpy.mean([containment_of_sentence_1_in_2, containment_of_sentence_2_in_1])

def JaccardCoefficient(sent_1_tokens, sent_2_tokens, ngram_size=1, character_ngram=False):
  set1, set2 = ngram_keys(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram)
  return float(len(set1.intersection(set2)))/len(set1.union(set2))

def POSTags_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1):
  postag_tokens_1 = process.postags(sent_1_tokens)
  postag_tokens_2 = process.postags(sent_2_tokens)
  return JaccardCoefficient(postag_tokens_1, postag_tokens_2, ngram_size), containment_coefficienct(postag_tokens_1, postag_tokens_2, ngram_size)

def Lemma_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1):
  lemma_tokens_1 = process.lemmatize(sent_1_tokens)
  lemma_tokens_2 = process.lemmatize(sent_2_tokens)
  return JaccardCoefficient(lemma_tokens_1, lemma_tokens_2, ngram_size), containment_coefficienct(lemma_tokens_1, lemma_tokens_2, ngram_size)

def character_ngram_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1, character_ngram=True):
  return JaccardCoefficient(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram), containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram)  


In [75]:
#print ngram_keys(["sent_1_tokens","test"], ["sent_2_tokens","tes","Tesd","Tesd","Tesd"], ngram_size=2,character_ngram=False)
print containment_coefficienct(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
#print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
#print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=2)
#print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=3)
#print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=4)

set([('is',), ('a',), ('rose',)]) set([('is',), ('a',), ('flower',), ('which',), ('rose',)])
0.8


In [76]:
sent_1_tokens, sent_2_tokens = ["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"]
character_ngram_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=2, character_ngram=True)
POSTags_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=2)

set([('o', 's'), ('s', 'e'), ('r', 'o'), ('i', 's')]) set([('f', 'l'), ('i', 'c'), ('e', 'r'), ('l', 'o'), ('h', 'i'), ('o', 's'), ('s', 'e'), ('r', 'o'), ('w', 'e'), ('w', 'h'), ('c', 'h'), ('i', 's'), ('o', 'w')])
set([('o', 's'), ('s', 'e'), ('r', 'o'), ('i', 's')]) set([('f', 'l'), ('i', 'c'), ('e', 'r'), ('l', 'o'), ('h', 'i'), ('o', 's'), ('s', 'e'), ('r', 'o'), ('w', 'e'), ('w', 'h'), ('c', 'h'), ('i', 's'), ('o', 'w')])
set([(('rose', 'VBD'), ('is', 'VBZ')), (('a', 'DT'), ('rose', 'VBD')), (('is', 'VBZ'), ('a', 'DT'))]) set([(('a', 'DT'), ('rose', 'VBD')), (('is', 'VBZ'), ('a', 'DT')), (('rose', 'VBD'), ('is', 'VBZ')), (('which', 'WDT'), ('is', 'VBZ')), (('a', 'DT'), ('flower', 'NN')), (('flower', 'NN'), ('which', 'WDT'))])
set([(('rose', 'VBD'), ('is', 'VBZ')), (('a', 'DT'), ('rose', 'VBD')), (('is', 'VBZ'), ('a', 'DT'))]) set([(('a', 'DT'), ('rose', 'VBD')), (('is', 'VBZ'), ('a', 'DT')), (('rose', 'VBD'), ('is', 'VBZ')), (('which', 'WDT'), ('is', 'VBZ')), (('a', 'DT'), ('flow

(0.5, 0.75)

In [77]:
TFIDFScores, Vocabulary, DocVectors, IDFVector = TFIDF(["i ike", "i ike", "i test", "hi ike"])
print TFIDFScores
x = [(1,2,3,4),(1,2,13,4)]
[ (t[2]) for t in x]
Counter(set([1,1,2]))


defaultdict(<function <lambda> at 0x116cffb90>, {'i': 0.0, 'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'ike': 0.0})


Counter({1: 1, 2: 1})

In [78]:
def CharacterIDFVector(documents, ngram_size=2):
  No_of_Documents = float(len(documents))
  IDFVector = Counter()
  for document in documents:
    tokens = process.tokens(document)
    IDFVector += Counter(set(character_ngram_vector_keys(tokens, ngram_size)))
  #print IDFVector
  for key in IDFVector.keys():
    IDFVector[key] = math.log(No_of_Documents/(1+IDFVector[key]))
  return IDFVector

def ngram_vector_keys(tokens, ngram_size=1):
  vector_keys = []
  for i in range(len(tokens)-(ngram_size-1)):
    vector_keys.append(tuple(tokens[i:i+(ngram_size)]))
    # vector_keys.append(" ".join(tokens[i:i+(ngram_size)]))
  return vector_keys

def character_ngram_vector_keys(tokens, ngram_size=2):
  vector_keys = []
  for token in tokens:
    for i in range(len(token)-(ngram_size-1)):
      vector_keys.append(tuple(token[i:i+(ngram_size)]))
  return vector_keys

def ngram_keys(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram):
  if character_ngram:
    set1 = set(character_ngram_vector_keys(sent_1_tokens, ngram_size))
    set2 = set(character_ngram_vector_keys(sent_2_tokens, ngram_size))
  else:
    set1 = set(ngram_vector_keys(sent_1_tokens, ngram_size))
    set2 = set(ngram_vector_keys(sent_2_tokens, ngram_size))
  return set1, set2

def ngram_weighted_value(keys, IDFScores, character_ngram=False):
  value = 0.0
  if not character_ngram:
    for key in keys:
      for i in range(len(key)):# This is a tuple
        if key[i] in IDFScores.keys():
          value += IDFScores[key[i]]
  else:
    for key in keys:
      if key in IDFScores.keys():
        value+=IDFScores[key]
  return value


def similarity_score(set1, set2, IDFScores=None, character_ngram=False, ngram_weighing=False):
  if not ngram_weighing:
    numerator = len(set1)
    denominator = len(set2)
  else:
    print set1
    print set2
    print character_ngram
    print IDFScores
    numerator = ngram_weighted_value(set1, IDFScores, character_ngram)
    denominator = ngram_weighted_value(set2, IDFScores, character_ngram)
    # INCASE IDF Values are not present for it
    if denominator == 0:
      numerator = len(set1)
      denominator = len(set2)
  return numerator/float(denominator)

def containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1, character_ngram=False, ngram_weighing=False, IDFScores=None):
  set1, set2 = ngram_keys(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram)
  intersecting_keys = (set1.intersection(set2))
  containment_of_sentence_1_in_2 = similarity_score(intersecting_keys, set1, IDFScores, character_ngram, ngram_weighing)
  containment_of_sentence_2_in_1 = similarity_score(intersecting_keys, set2, IDFScores, character_ngram, ngram_weighing)
  # return containment_of_sentence_1_in_2, containment_of_sentence_2_in_1
  return numpy.mean([containment_of_sentence_1_in_2, containment_of_sentence_2_in_1])

def JaccardCoefficient(sent_1_tokens, sent_2_tokens, ngram_size=1, character_ngram=False, ngram_weighing=False, IDFScores=None):
  set1, set2 = ngram_keys(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram)
  
  return similarity_score(set1.intersection(set2), set1.union(set2), IDFScores, character_ngram, ngram_weighing)
  # return float(len(set1.intersection(set2)))/len(set1.union(set2))

def POSTags_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1, ngram_weighing=False, IDFScores=None):
  postag_tokens_1 = process.postags(sent_1_tokens)
  postag_tokens_2 = process.postags(sent_2_tokens)
  return JaccardCoefficient(postag_tokens_1, postag_tokens_2, ngram_size, False, ngram_weighing, IDFScores), containment_coefficienct(postag_tokens_1, postag_tokens_2, ngram_size, False, ngram_weighing, IDFScores)

def Lemma_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1, ngram_weighing=False, IDFScores=None):
  lemma_tokens_1 = process.lemmatize(sent_1_tokens)
  lemma_tokens_2 = process.lemmatize(sent_2_tokens)
  return JaccardCoefficient(lemma_tokens_1, lemma_tokens_2, ngram_size, False, ngram_weighing, IDFScores), containment_coefficienct(lemma_tokens_1, lemma_tokens_2, ngram_size, False, ngram_weighing, IDFScores)

def character_ngram_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=2, character_ngram=True, ngram_weighing=False, IDFScores=None):
  return JaccardCoefficient(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram, ngram_weighing, IDFScores), containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size, character_ngram, ngram_weighing, IDFScores)

In [79]:
IDFScores = CharacterIDFVector(["i ike", "hi tom","test"],2)
print IDFScores
TFIDFScores, Vocabulary, DocVectors, IDFVector = TFIDF(["i ike", "i ike", "i test", "hi ike"])
print IDFVector
tok1=["i","test"]
tok2=["i","rest"]
print Lemma_JaccardCoefficient_and_containment_coefficienct(tok1, tok2, ngram_size=1, ngram_weighing=True, IDFScores=IDFVector)
print character_ngram_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, 2, True, True, IDFScores)
print character_ngram_JaccardCoefficient_and_containment_coefficienct(tok1, tok2, 2, True, True, IDFScores)



Counter({('t', 'o'): 0.4054651081081644, ('k', 'e'): 0.4054651081081644, ('e', 's'): 0.4054651081081644, ('h', 'i'): 0.4054651081081644, ('i', 'k'): 0.4054651081081644, ('o', 'm'): 0.4054651081081644, ('t', 'e'): 0.4054651081081644, ('s', 't'): 0.4054651081081644})
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
set([('i',)])
set([('test',), ('i',), ('rest',)])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
set([('i',)])
set([('test',), ('i',)])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
set([('i',)])
set([('i',), ('rest',)])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
(0.0, 0.25)
set([('o', 's'), ('s', 'e'), ('r', 'o'), ('i', 's')])
set([('f', 'l'), ('i', 'c'), ('h', 'i'), ('s', 'e'), ('w', 'e'), ('w', 'h'), ('o', 'w'), ('c', 'h'), ('e', 'r'), ('l', 'o'), ('o', 's'), ('r', 'o'), ('i', 's')])
True
Counter({('

In [80]:
print containment_coefficienct(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=2)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=3)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=4)


0.8
0.6
0.5
0.428571428571
0.125


In [81]:
tok1=["hi","test"]
tok2=["hi","test", "this"]
print POSTags_JaccardCoefficient_and_containment_coefficienct(tok1, tok2, 2, True, IDFVector)

set([(('hi', 'JJ'), ('test', 'NN'))])
set([(('test', 'NN'), ('this', 'DT')), (('hi', 'JJ'), ('test', 'NN'))])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
set([(('hi', 'JJ'), ('test', 'NN'))])
set([(('hi', 'JJ'), ('test', 'NN'))])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
set([(('hi', 'JJ'), ('test', 'NN'))])
set([(('test', 'NN'), ('this', 'DT')), (('hi', 'JJ'), ('test', 'NN'))])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
(0.5, 0.75)


In [82]:
print Lemma_JaccardCoefficient_and_containment_coefficienct(tok1, tok2, ngram_size=1, ngram_weighing=True, IDFScores=IDFVector)

set([('test',), ('hi',)])
set([('this',), ('test',), ('hi',)])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
set([('test',), ('hi',)])
set([('test',), ('hi',)])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
set([('test',), ('hi',)])
set([('this',), ('test',), ('hi',)])
False
Counter({'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0})
(1.0, 1.0)


In [83]:
print character_ngram_JaccardCoefficient_and_containment_coefficienct(tok1, tok1, 2, True, True, IDFScores)

set([('e', 's'), ('h', 'i'), ('t', 'e'), ('s', 't')])
set([('e', 's'), ('h', 'i'), ('t', 'e'), ('s', 't')])
True
Counter({('t', 'o'): 0.4054651081081644, ('k', 'e'): 0.4054651081081644, ('e', 's'): 0.4054651081081644, ('h', 'i'): 0.4054651081081644, ('i', 'k'): 0.4054651081081644, ('o', 'm'): 0.4054651081081644, ('t', 'e'): 0.4054651081081644, ('s', 't'): 0.4054651081081644})
set([('e', 's'), ('h', 'i'), ('t', 'e'), ('s', 't')])
set([('e', 's'), ('h', 'i'), ('t', 'e'), ('s', 't')])
True
Counter({('t', 'o'): 0.4054651081081644, ('k', 'e'): 0.4054651081081644, ('e', 's'): 0.4054651081081644, ('h', 'i'): 0.4054651081081644, ('i', 'k'): 0.4054651081081644, ('o', 'm'): 0.4054651081081644, ('t', 'e'): 0.4054651081081644, ('s', 't'): 0.4054651081081644})
set([('e', 's'), ('h', 'i'), ('t', 'e'), ('s', 't')])
set([('e', 's'), ('h', 'i'), ('t', 'e'), ('s', 't')])
True
Counter({('t', 'o'): 0.4054651081081644, ('k', 'e'): 0.4054651081081644, ('e', 's'): 0.4054651081081644, ('h', 'i'): 0.4054651081

In [84]:


def main():
  Data = data_collection.Data();
  #Data = Data[:10]
  corpus_size = len(Data)
  print "The Total Corpus Data is about " + str(corpus_size*2)
  #cosine_similarity_without_tfidf_predicted_answers = cosine_similarity_without_tfidf(Data)
  train_split = 0.8
  training_data_documents_size = int(round(corpus_size * train_split))
  #print training_data_documents_size
  test_data_documents_size = corpus_size - training_data_documents_size
  training_documents =  Data[:training_data_documents_size]
  test_documents = Data[training_data_documents_size+1:]
  print "Training on " + str(training_data_documents_size*2) + " documents"
  print "Testing on " + str(test_data_documents_size) + " documents"
  training_documents_answers = [item[2] for item in training_documents]
  training_documents = [item[0:2] for item in training_documents]
  training_documents = list(itertools.chain.from_iterable(training_documents))
  test_documents_answers = [item[2] for item in test_documents]
  test_documents = [item[0:2] for item in test_documents]
  test_documents = list(itertools.chain.from_iterable(test_documents))
  train_and_test_simple_model(training_documents, test_documents, training_documents_answers, test_documents_answers, load=True)


def cosine_similarity_without_tfidf(documents):
  answers = []
  predicted_answers = []
  ind=0
  for document in documents:
    answers.append(document[2])
    predicted_answers.append(5 * ngram.cosinesimilarity_without_TFIDF(document[0], document[1]))
    print answers[ind],   "   |   ",  predicted_answers[ind]
    ind+=1
  print "Error in Estimate is " + str(evaluate(answers, predicted_answers))
  return predicted_answers

def evaluate(gold_standard, predicted_answers):
  return np.sum(np.absolute(np.array(gold_standard) - np.array(predicted_answers)))/len(gold_standard) 
  # return math.sqrt(np.sum(np.power(np.array(gold_standard) - np.array(predicted_answers),2))) 

def train_and_test_simple_model(train_documents, test_documents, training_documents_answers, test_documents_answers, load=False):
  if(not load):
    TFIDFScores, Vocabulary, DocVectors, IDFVector = ngram.TFIDF(train_documents)
    #print TFIDFScores
    utility.save_weights("TFIDFScores.dat",TFIDFScores);
    utility.save_weights("Vocabulary.dat", Vocabulary);
    utility.save_weights("DocVectors.dat", DocVectors);
    utility.save_weights("IDFVector.dat", IDFVector);
  else:
    TFIDFScores=utility.load_weights("weights/TFIDFScores.dat")
    #print TFIDFScores
    Vocabulary=utility.load_weights("weights/Vocabulary.dat")
    DocVectors=utility.load_weights("weights/DocVectors.dat")
    IDFVector=utility.load_weights("weights/IDFVector.dat")
  f= open("analysis.txt","w")
  print "Training Documents Analysis"
  print "-------------------------------------------------------------------"
  #train_predicted_answers = cosinesimilarity_evaluate_TFIDF(train_documents, TFIDFScores, training_documents_answers)
  
  print "Test Documents Analysis"
  print "-------------------------------------------------------------------"
  #test_predicted_answers = cosinesimilarity_evaluate_TFIDF(test_documents, TFIDFScores, test_documents_answers)
  for ngram_size in range(1,2):
    for analyze_type in ["lemma"]:#, "character"]:
      if(analyze_type=="character"):
        if(not load):
          idf_scores = ngram.CharacterIDFVector(train_documents, ngram_size)
          utility.save_weights("IDF_Char_"+str(ngram_size)+"_gram.dat", idf_scores)
        else:
          idf_scores = utility.load_weights("weights/IDF_Char_"+str(ngram_size)+"_gram.dat")
      else:
        idf_scores = IDFVector
      # print "-------------------------------------------------------------------"
      # f.write("-------------------------------------------------------------------")
      # print "Jaccards and Containment Coefficient Analysis without using ngram weighing and type = " + analyze_type + " ngram = " + str(ngram_size)
      # f.write("Jaccards and Containment Coefficient Analysis without using ngram weighing and type = " + analyze_type + " ngram = " + str(ngram_size))
      # print "-------------------------------------------------------------------"
      # f.write("-------------------------------------------------------------------")
      # Jacc_one_gram_pred_answers, Containment_one_gram_pred_answers = jaccard_and_containment_coefficient_evaluate(analyze_type, train_documents, training_documents_answers, ngram_size, False, IDFScores=None)
      # Jacc_one_gram_pred_answers, Containment_one_gram_pred_answers = jaccard_and_containment_coefficient_evaluate(analyze_type, test_documents, test_documents_answers, ngram_size, False, IDFScores=None)
      print "-------------------------------------------------------------------"
      f.write("-------------------------------------------------------------------")
      print "Jaccards and Containment Coefficient Analysis using ngram weighing and type = " + analyze_type + " ngram = " + str(ngram_size)
      f.write("Jaccards and Containment Coefficient Analysis using ngram weighing and type = " + analyze_type + " ngram = " + str(ngram_size))
      print "-------------------------------------------------------------------"
      f.write("-------------------------------------------------------------------")
      #train_documents, training_documents_answers = ["TEst this one","Test this two"],[3.0]
      #test_documents, test_documents_answers = ["TEst this one","Test this two"],[3.0]
      Jacc_one_gram_pred_answers, Containment_one_gram_pred_answers = jaccard_and_containment_coefficient_evaluate(analyze_type, train_documents, training_documents_answers, ngram_size, True, idf_scores)
      Jacc_one_gram_pred_answers, Containment_one_gram_pred_answers = jaccard_and_containment_coefficient_evaluate(analyze_type, test_documents, test_documents_answers, ngram_size, True, idf_scores)
  f.close()

def cosinesimilarity_evaluate_TFIDF(documents, TFIDFScores, answers):
  ind = 0
  predicted_answers = []
  for i in range(len(documents)/2):
    document1, document2 = documents[(2*i)], documents[(2*i)+1]
    #print document1, document2
    predicted_answers.append(5*ngram.cosinesimilarity(document1, document2, TFIDFScores))
    #print answers[ind],   "   |   ", predicted_answers[ind]
    ind+=1
  print "Error in Estimate is " + str(evaluate(answers, predicted_answers))
  return predicted_answers 

def jaccard_and_containment_coefficient_evaluate(analyze_type, documents, answers, ngram_size=1, ngram_weighing=False, IDFScores=None):
  ind = 0
  containment_coefficient_predicted_answers = []
  jaccard_coefficient_predicted_answers = []
  for i in range(len(documents)/2):
    document1, document2 = documents[(2*i)], documents[(2*i)+1]
    #print document1, document2
    sent_1_tokens = process.tokens(document1)
    sent_2_tokens = process.tokens(document2)
    if(analyze_type=="pos"):
      jaccard_coefficient, containment_coefficient = ngram.POSTags_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size, ngram_weighing, IDFScores)
    elif(analyze_type=="lemma"):
      jaccard_coefficient, containment_coefficient = ngram.Lemma_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size, ngram_weighing, IDFScores)
    elif(analyze_type=="character"):
      jaccard_coefficient, containment_coefficient = ngram.character_ngram_JaccardCoefficient_and_containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size, True, ngram_weighing, IDFScores)
    jaccard_coefficient_predicted_answers.append(5*jaccard_coefficient)
    containment_coefficient_predicted_answers.append(5*containment_coefficient)
    #print answers[ind],   "   |   ", jaccard_coefficient_predicted_answers[ind]
    #print answers[ind],   "   |   ", containment_coefficient_predicted_answers[ind]
    ind+=1
  print "Error in Estimate For Jaccard Coefficient is " + str(evaluate(answers, jaccard_coefficient_predicted_answers))
  print "Error in Estimate For Containment Coefficient is " + str(evaluate(answers, containment_coefficient_predicted_answers))
  return jaccard_coefficient_predicted_answers, containment_coefficient_predicted_answers


In [85]:
def ngram_weighted_value(keys, IDFScores, character_ngram=False):
  value = 0.0
  if not character_ngram:
    for key in keys:
      for i in range(len(key)):# key is a tuple
        for j in range(len(key[i])):
          print key[i][j]
          if key[i][j] in IDFScores.keys():
            value += IDFScores[key[i][j]]
  else:
    for key in keys:
      if key in IDFScores.keys():
        value+=IDFScores[key]
  return value

def ngram_weighted_value(keys, IDFScores, character_ngram=False):
  value = 0.0
  if not character_ngram:
    for key in keys:
      for i in range(len(key)):# This is a tuple
        if(type(key[i]) is not tuple):
          print key[i]
          if key[i] in IDFScores.keys():
            value += IDFScores[key[i]]
        else:
          print "dan"
          print key[i][0]
          if key[i][0] in IDFScores.keys():
            value += IDFScores[key[i][0]] 
  else:
    for key in keys:
      if key in IDFScores.keys():
        value+=IDFScores[key]
  return value
print tok1, tok2
postag_tokens_1 = process.lemmatize(tok1)
postag_tokens_2 = process.lemmatize(tok2)
print postag_tokens_1,postag_tokens_2
IDFScores = {'test': 0.6931471805599453, 'hi': 0.6931471805599453, 'i': 0.0, 'ike': 0.0}
set1,set2= ngram_keys(postag_tokens_1,postag_tokens_2, 2,False)
print set1, set2
ngram_weighted_value(set2, IDFScores, False)


['hi', 'test'] ['hi', 'test', 'this']
['hi', 'test'] ['hi', 'test', 'this']
set([('hi', 'test')]) set([('hi', 'test'), ('test', 'this')])
hi
test
test
this


2.0794415416798357

In [110]:
# create X and y
feature_cols = ['TV', 'Radio', 'Newspaper']
X = [[1,1,1],[2,2,2],[3,3,3],]
y = [1,2,3]

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print lm.intercept_
print lm.coef_
lm.predict([[4, 4, 4],[5, 5, 5]])
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=True)
Corpus = ["This is a test run","Test this out i"]
D = [{'foo': 2, 'bar': 2}, {'foo': 3, 'baz': 1}]
lm = LinearRegression()
X = v.fit_transform(D)
lm.fit(X, [1,0.5])
# print intercept and coefficients
print lm.intercept_
print lm.coef_
D2 = [{'foo': 1, 'bar': 3}, {'foo': 3, 'baz': 1},{'sss':1,'sdasda':1}]
pX = v.transform(D2)
print lm.predict(pX)

print Counter(['1','2','1']) + Counter(['1','2','0'])

1.11022302463e-15
[ 0.33333333  0.33333333  0.33333333]
0.75
[ 0.18181818 -0.07954545 -0.05681818]
[ 1.23863636  0.5         0.75      ]
Counter({'1': 3, '2': 2, '0': 1})
